In [3]:
import polars as pl
import datashader as ds
import datashader.transfer_functions as tf
import matplotlib.pyplot as plt

file = 'english'

# Leggi il primo file con Polars
df1 = pl.read_csv(f'delays_txt/{file}_delays.txt', has_header=False, dtypes={"column_1": pl.Float64})
df1 = df1.rename({'column_1': 'y'})

# Calcola il valore massimo e minimo della colonna 'y' del primo dataset
y_min1 = df1['y'].min()
y_max1 = df1['y'].max()

# Aggiungi la colonna 'x' al primo DataFrame
df1 = df1.with_columns([
    pl.Series(name='x', values=range(len(df1)))
])

# Riduci il primo DataFrame
step = 10
df_reduced1 = df1[::step]

# Calcola la media per il ridotto DataFrame 1
threshold1 = df_reduced1['y'].mean()

# Ridimensiona i valori della colonna 'y' del primo DataFrame
df_reduced1 = df_reduced1.with_columns([
    pl.when(df_reduced1['y'] > threshold1).then(df_reduced1['y'] / 20).otherwise(df_reduced1['y']).alias('y_adjusted')
])

# Leggi il secondo file con Polars
file2 = 'english'
df2 = pl.read_csv(f'delays_txt/{file2}_delays.txt', has_header=False, dtypes={"column_1": pl.Float64})
df2 = df2.rename({'column_1': 'y'})

# Calcola il valore massimo e minimo della colonna 'y' del secondo dataset
y_min2 = df2['y'].min()
y_max2 = df2['y'].max()

# Ridimensiona i valori della colonna 'y' del secondo DataFrame utilizzando la scala del primo DataFrame
df2 = df2.with_columns([
    pl.when(df2['y'] > threshold1).then(df2['y'] / 20).otherwise(df2['y']).alias('y_adjusted')
])

# Aggiungi la colonna 'x' al secondo DataFrame
df2 = df2.with_columns([
    pl.Series(name='x', values=range(len(df2)))
])

# Riduci il secondo DataFrame
df_reduced2 = df2[::step]

# Conversione a DataFrame Pandas per l'uso con Datashader
df_pandas2 = df_reduced2.to_pandas()

# Descrizione del DataFrame ridotto (opzionale)
print(df_pandas2.describe())

canvas = ds.Canvas(plot_width=2000, plot_height=1000)
agg = canvas.line(df_pandas2, 'x', 'y_adjusted', ds.count())

img = tf.shade(agg)
img = tf.set_background(img, "white").to_pil()

plt.figure(figsize=(20, 5))
# Usa gli estremi della scala del primo dataset per mantenere la stessa scala delle y
plt.imshow(img, aspect='auto', extent=(0, len(df2) - 1, y_min1, y_max1), origin='upper')
plt.ylim(y_min1, y_max1)

plt.savefig(f'delays_png/{file2}_delays_std.png', bbox_inches='tight', pad_inches=0.1)
plt.show()


/var/folders/s6/rbq2n_1j59ncwkjkh13j8rlc0000gn/T/ipykernel_22293/870788312.py:9: DeprecationWarning: The argument `dtypes` for `read_csv` is deprecated. It has been renamed to `schema_overrides`.
  df1 = pl.read_csv(f'delays_txt/{file}_delays.txt', has_header=False, dtypes={"column_1": pl.Float64})
/var/folders/s6/rbq2n_1j59ncwkjkh13j8rlc0000gn/T/ipykernel_22293/870788312.py:35: DeprecationWarning: The argument `dtypes` for `read_csv` is deprecated. It has been renamed to `schema_overrides`.
  df2 = pl.read_csv(f'delays_txt/{file2}_delays.txt', has_header=False, dtypes={"column_1": pl.Float64})


FileNotFoundError: No such file or directory (os error 2): delays_txt/dblp_delays.txt